In [ ]:
import numpy as np
import scipy.ndimage as spndim
import pandas as pd
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import matplotlib.cm as mcm
import matplotlib.colorbar as mcb
import matplotlib.ticker as mtk
import utils.engine as ue
import utils.visualizer as uv

In [ ]:
%load_ext autoreload
%load_ext line_profiler
%autoreload 2

In [ ]:
plt.rcParams["animation.html"] = "jshtml"

In [ ]:
# with open("Validation/SHEBA_Data/01-forcing.dat") as file:
#     arr = file.readlines()
    
# cols_1 = [colname.replace('\n', '') for colname in arr[0].split(" ") if colname]
# file_1 = np.array([[float(el) for el in line.split(" ") if el] for line in arr[1:]])

In [ ]:
# with open("Validation/SHEBA_Data/02-hs-hi_thickness.dat") as file:
#     arr = file.readlines()
    
# cols_2 = [colname.replace('\n', '') for colname in arr[0].split(" ") if colname]
# file_2 = np.array([[float(el) for el in line.split(" ") if el] for line in arr[1:]])

In [ ]:
# with open("Validation/SHEBA_Data/06-depth_pit.dat") as file:
#     arr = file.readlines()
    
# file_6 = np.array([[float(el) for el in line.split(" ") if el] for line in arr])
# file_6[:, 0] = np.round(file_6[:, 0], 4)

In [ ]:
# with open("Validation/SHEBA_Data/07-pit_fcond.dat") as file:
#     arr = file.readlines()
    
# cols_7 = [colname.replace('\n', '') for colname in arr[0].split(" ") if colname]
# file_7 = np.array([[float(el) for el in line.split(" ") if el] for line in arr[1:]])

In [ ]:
# with open("Validation/SHEBA_Data/08-tocn.dat") as file:
#     arr = file.readlines()
    
# file_8 = np.array([[float(el) for el in line.split(" ") if el] for line in arr])

In [ ]:
# with open("Validation/SHEBA_Data/09-focn.dat") as file:
#     arr = file.readlines()
    
# cols_9 = [colname.replace('\n', '') for colname in arr[0].split(" ") if colname]
# file_9 = np.array([[float(el) for el in line.split(" ") if el] for line in arr[1:]])

In [ ]:
# with open("Validation/SHEBA_Data/10-temp.dat") as file:
#     arr = file.readlines()
    
# file_10 = np.array([[float(el) for el in line.split(" ") if el] for line in arr])

In [ ]:
# with open("Validation/SHEBA_Data/11-turbflux.dat") as file:
#     arr = file.readlines()
    
# cols_11 = [colname.replace('\n', '') for colname in arr[0].split(" ") if colname]
# file_11 = np.array([[float(el) for el in line.split(" ") if el] for line in arr[1:]])

In [ ]:
# data = pd.DataFrame(file_1, columns=cols_1)

# data_2 = pd.DataFrame(file_2, columns=cols_2)
# data = data.merge(data_2.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_2['jday']*1000).astype('int32')).iloc[:, 1:]

# data_6 = pd.DataFrame(file_6, columns=['jday', 'hs', 'hi', 'hs_level', 'hf_level', 'hi_level'])
# data = data.merge(data_6.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_6['jday']*1000).astype('int32')).iloc[:, 1:]

# data_7 = pd.DataFrame(file_7, columns=cols_7)
# data = data.merge(data_7.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_7['jday']*1000).astype('int32')).iloc[:, 1:]

# data_8 = pd.DataFrame(file_8, columns=['jday', 'Tocn'])
# data = data.merge(data_8.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_8['jday']*1000).astype('int32')).iloc[:, 1:]

# data_9 = pd.DataFrame(file_9, columns=cols_9)
# data = data.merge(data_9.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_9['jday']*1000).astype('int32')).iloc[:, 1:]

# data_10 = pd.DataFrame(file_10, columns=['jday'] + ['T_{}'.format(i) for i in range(1, 51)])
# data = data.merge(data_10.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_10['jday']*1000).astype('int32')).iloc[:, 1:]

# data_11 = pd.DataFrame(file_11, columns=cols_11)
# data = data.merge(data_11.iloc[:, 1:],
#                   left_on=(data['jday']*1000).astype('int32'),
#                   right_on=(data_11['jday']*1000).astype('int32')).iloc[:, 1:]

In [ ]:
# data.to_csv('Validation/full_data.csv', index=False)

In [ ]:
with open("Validation/SHEBA_Data/12-levels.dat") as file:
    Z = [float(z.replace(' ', '')[:-1]) for z in file.readlines()]

In [ ]:
data = pd.read_csv('Validation/full_data.csv')

In [ ]:
data.columns

In [ ]:
prec = 5.
cmap_base = plt.get_cmap('rainbow')
vmin = data.loc[:, 'T_1':'T_50'].values.min()
vmax = data.loc[:, 'T_1':'T_50'].values.max()

boundaries = np.concatenate(([vmin], np.arange(np.floor(vmin + prec), vmax, prec), [vmax]))
centers = (boundaries[1:] + boundaries[:-1])/2
centers_scaled = (centers - vmin) / (vmax - vmin)

cmap_custom = clr.ListedColormap([cmap_base(x) for x in centers_scaled])
norm_custom = clr.BoundaryNorm(boundaries, cmap_custom.N)

In [ ]:
plt.figure(figsize=(20, 10))
plt.imshow(data.loc[:, 'T_1':'T_50'].T,
           aspect='auto', cmap=cmap_custom, norm=norm_custom,
           extent=[data.loc[0, 'jday'], data.loc[data.index[-1], 'jday'], Z[-1], Z[0]])
plt.plot(data['jday'], data['hs_level'], color='white', lw=3, ls=':', label='snow surface')
plt.plot(data['jday'], data['hf_level'], color='white', lw=3, ls='--', label='snow-ice interface')
plt.plot(data['jday'], data['hi_level'], color='white', lw=3, label='ice base')
plt.plot(data['jday'], data['hs_best'], color='black', lw=3, ls=':', label='snow surface best')
plt.xticks(size=15)
plt.yticks(size=15)
plt.legend(loc='lower left', facecolor='black', labelcolor='white', prop={'size': 20})
plt.colorbar(ticks=boundaries)
# plt.savefig('test-image.png', dpi=150)
plt.show()

Судя по статье (Fig.5 (a), (b)), чёрные точки - это метод приближения, использованный в статье Хувальда

In [ ]:
# sigma mesh
dsigma_ice = np.full(20, 1.0/20)
dsigma_snow = np.full(5, 1.0/5)

In [ ]:
# init state from data
T_ice_init, T_snow_init, Toi_init, Tis_init, Tsa_init, \
dzi_init, dzs_init = ue.get_init_from_data(data, Z, dsigma_ice, dsigma_snow)

time_arr = ((data['jday'] - data.loc[0, 'jday'])*24*3600).values

In [ ]:
# process from data
process_data = ue.process_from_data(Z,
                                 data.loc[:, 'T_1':'T_50'].values, 
                                 data['Tib_interp'].values, data['Tis_interp'].values,
                                 data['Tss'].values,
                                 data['hi_best'].values, data['hf_best'].values,
                                 data['hs_best'].values,
                                 np.ones(20)/20.0, np.ones(5)/5.0,
                                 time_arr, ue.rho_i)

In [ ]:
# forcing
e_s = lambda T: 6.11*np.exp(ue.c1_i*T/(T + ue.T0 - ue.c2_i))
q_surf = lambda T: 0.622*e_s(T)/(ue.P_surf - 0.378*e_s(T))

albe = data['albe'].values
Tib_interp = data['Tib_interp'].values
tair = data['tair'].values
uair = data['uair'].values
qair = data['qair'].values
prec = data['prec_eff'].values
swdo = data['swdo'].values
lwdo = data['lwdo'].values
focn_pit = data['focn_pit'].values
Focn_y = data['Focn_y'].values


find_index_fast = lambda time_sec: np.searchsorted(time_arr, time_sec, side='right') - 1
albedo = lambda time: albe[find_index_fast(time)]
Toi = lambda time: Tib_interp[find_index_fast(time)]
T_a = lambda time: tair[find_index_fast(time)] - ue.T0
u_a = lambda time: uair[find_index_fast(time)]
q_a = lambda time: qair[find_index_fast(time)] * 1e-3
p = lambda time: prec[find_index_fast(time)] * ue.mmd_to_ms
F_sw = lambda time: swdo[find_index_fast(time)]
F_lw = lambda time: lwdo[find_index_fast(time)]
F_bolz = lambda T: -ue.sigma*(T + ue.T0)**4
F_sh = lambda T, time: ue.rho_a*ue.c_pa*ue.C_sh*u_a(time)*(T_a(time) - T)
F_lh = lambda T, time: ue.rho_a*ue.L_s0*ue.C_lh*u_a(time)*(q_a(time) - q_surf(T))
F_P = lambda T, time: p(time)*ue.rho_w*ue.c_pw*max(T_a(time) - T, 0)
F_atm = lambda T, time, i_0: ue.emissivity*(F_lw(time) + F_bolz(T)) \
                           + (1 - albedo(time))*(1 - i_0)*F_sw(time) \
                           + F_sh(T, time) + F_P(T, time) + F_lh(T, time) 

F_atm_ice = lambda T, time: F_atm(T, time, ue.i0_i)
F_atm_snow = lambda T, time: F_atm(T, time, ue.i0_s)
F_ocn = lambda T, time: Focn_y[find_index_fast(time)]

In [ ]:
%%time
process_sim = ue.main_process \
(
    time_step=3600.0,
    time_end=7774*3600.0,
    N_pseudoiter=50,
    Ti_init=T_ice_init,
    Ts_init=T_snow_init,
    Tis_init=Tis_init,
    Tsa_init=Tsa_init,
    dzi_init=dzi_init,
    dzs_init=dzs_init,
    salinity=np.linspace(4.0, 1.0, len(dzi_init)),
    snow_thickness_threshold=0.01,
    Toi=Toi,
    Ta=T_a,
    p=p,
    F_atm_ice=F_atm_ice,
    F_atm_snow=F_atm_snow,
    F_sw=F_sw,
    F_ocn=F_ocn
)

In [ ]:
test_anim = uv.animate([process_data], ue.rho_w, ue.rho_s, clip_end=50)

In [ ]:
test_anim

In [ ]:
out = uv.timeseries_img(process_data, y_points=100)#, savepath='timeseries.png')

In [ ]:
uv.timeseries_img(process_data, y_points=200,
                  cmap_ice='RdBu', tmin_ice=-25, tmax_ice=2, step_ice=1.0,
                  cmap_snow='RdBu', tmin_snow=-25, tmax_snow=2, step_snow=1.0)

In [ ]:
# rho_water=ue.rho_w
# rho_snow=ue.rho_s
# y_points=100
# t_min=None
# t_max=None
# rgb_background = [208, 245, 226]
# levels_fill = np.arange(-6, 6.1, 0.5)
# levels_border = [-3, -2, -1, -0.5, 0.5, 1, 2]
# border_intense = 4
# sigma_x = 0.0
# sigma_y = 15.0
# figsize=(30, 10)
# cmap='seismic'

# Z_i_data, Z_s_data = uv.get_Z(process_data, rho_snow, rho_water)
# Z_i_sim, Z_s_sim = uv.get_Z(process_sim, rho_snow, rho_water)
# z_min = min(np.concatenate((Z_i_sim[:, 0], Z_i_data[:, 0])))
# z_max = max(np.concatenate((Z_s_sim[:, -1], Z_s_data[:, -1])))

# Z_mesh = np.linspace(z_min, z_max, y_points + 1)
# temp_mesh_data = np.array([np.interp(x=Z_mesh,
#                                      xp=np.concatenate((Z_i_line, (Z_s_line[1:] if has_snow else []))),
#                                      fp=np.concatenate(([T_oi], T_i_line, [T_is],
#                                                        (T_s_line if has_snow else []),
#                                                        ([T_sa] if has_snow else [])
#                                                        )),
#                                      left=np.nan, right=np.nan
#                                      ) \
#                       for Z_i_line, Z_s_line, T_oi, T_i_line, T_is, T_s_line, T_sa, has_snow \
#                       in zip(Z_i_data, Z_s_data,
#                              process_data.oi_temp_history, process_data.ice_temp_history,
#                              process_data.is_temp_history, process_data.snow_temp_history,
#                              process_data.sa_temp_history, process_data.snow_presence_history
#                             )
#                            ])
# temp_mesh_sim = np.array([np.interp(x=Z_mesh,
#                                     xp=np.concatenate((Z_i_line, (Z_s_line[1:] if has_snow else []))),
#                                     fp=np.concatenate(([T_oi], T_i_line, [T_is],
#                                                        (T_s_line if has_snow else []),
#                                                        ([T_sa] if has_snow else [])
#                                                        )),
#                                     left=np.nan, right=np.nan
#                                     ) \
#                       for Z_i_line, Z_s_line, T_oi, T_i_line, T_is, T_s_line, T_sa, has_snow \
#                       in zip(Z_i_sim, Z_s_sim,
#                              process_sim.oi_temp_history, process_sim.ice_temp_history,
#                              process_sim.is_temp_history, process_sim.snow_temp_history,
#                              process_sim.sa_temp_history, process_sim.snow_presence_history
#                             )
#                           ])

# data = (temp_mesh_sim - temp_mesh_data).T

# fig = plt.figure(figsize=figsize)
# ax = plt.axes()
# ax.set_facecolor(np.array(rgb_background)/256.0)
# contourf = ax.contourf(gauss_filter_with_nans(data, (sigma_x, sigma_y)),
#                        levels=levels_fill, cmap=cmap, extend='both',
#                        extent=[process_data.timeline[0]/3600, process_data.timeline[-1]/3600,
#                                Z_mesh[0], Z_mesh[-1]]
#                       )
# contour = ax.contour(gauss_filter_with_nans(data, (sigma_x, sigma_y)),
#                      levels=levels_border, cmap=cmap,
#                      norm=clr.Normalize(vmin=-2, vmax=2, clip=True),
#                      linewidths=2.5, linestyles='--',
#                      extent=[process_data.timeline[0]/3600, process_data.timeline[-1]/3600,
#                              Z_mesh[0], Z_mesh[-1]])
# ax.clabel(contour, fontsize=20)

# # ax.axhline(lw=3, ls='--', color='c')
# ax.plot(process_data.timeline/3600.0, Z_i_data[:, 0], lw=1.5, color='black')
# ax.plot(process_data.timeline/3600.0, Z_i_data[:, -1], lw=1.5, color='black')
# ax.plot(process_data.timeline/3600.0, Z_s_data[:, -1], lw=1.5, color='black', label='data')
# ax.plot(process_data.timeline/3600.0, Z_i_sim[:, 0], lw=3, ls=':', color='black')
# ax.plot(process_data.timeline/3600.0, Z_i_sim[:, -1], lw=3, ls=':', color='black')
# ax.plot(process_data.timeline/3600.0, Z_s_sim[:, -1], lw=3, ls=':', color='black', label='simulation')

# ax.set_xlabel('t, hours', size=20)
# ax.set_ylabel('Z, m', size=20)
# ax.tick_params(axis='both', labelsize=15)


# cax = fig.add_axes([ax.get_position().x0, ax.get_position().y0 - 0.15,
#                     ax.get_position().width, 0.05])
# colorbar = fig.colorbar(contourf, cax=cax, orientation='horizontal', ticks=contourf.levels, drawedges=True)
# colorbar.outline.set_linewidth(3)
# colorbar.dividers.set_linewidth(3)
# colorbar.dividers.set_dashes((-0.5, (2.5, 6.5)))
# cax_ice.tick_params(axis='x', labelsize=15)
# ax.legend(prop={'size':20})
# plt.show()

In [ ]:
uv.timeseries_err(process_sim, process_data)

In [ ]:
uv.timeseries_err(process_sim, process_data, tmin_err=-4., tmax_err=4., step_err=1.)

In [ ]:
uv.timeseries_err(process_sim[5000:6500], process_data[5000:6500], tmin_err=-4., tmax_err=4., step_err=.5)